# Introduction To Machine Learning
## Mahdi Tabatabaei (400101515) - Heliya Shakeri (400101391)
### Machine Unlearning

# **Installing Packages**

In [61]:
# Installing Libraries
! pip install tabulate

# **Importing Libraries**

In [59]:
# Importing Libraries
import torch
import torchvision
import torchvision.transforms as transforms
from torchvision.models import resnet18
import torch.nn as nn
import torch.nn.functional as F
from torch.utils.data import DataLoader, random_split, Subset, Dataset, TensorDataset, ConcatDataset
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score, roc_auc_score
from sklearn.preprocessing import label_binarize
from sklearn.linear_model import LogisticRegressionCV
from sklearn.model_selection import cross_val_score
from torch.nn.functional import cross_entropy
import numpy as np
import random
from tqdm import tqdm
import copy
from tabulate import tabulate
import warnings

warnings.filterwarnings('ignore')

# **Setup Device**

In [54]:
# Setup device
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
print(f'Using device: {device}')

Using device: cuda:0


# **Data Prepraration**

In [55]:
# Load and preprocess the dataset
def load_data():
    transform = transforms.Compose([
        transforms.Resize((224, 224)),
        transforms.ToTensor(),
        transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225]),
    ])
    train_set = torchvision.datasets.CIFAR10(root='./data', train=True, download=True, transform=transform)
    test_set = torchvision.datasets.CIFAR10(root='./data', train=False, download=True, transform=transform)
    return train_set, test_set

train_set, test_set = load_data()

Files already downloaded and verified

Files already downloaded and verified


# **Model Preparation**

In [63]:
# Modify ResNet-18 for CIFAR-10 (10 classes)
def create_model():
    model = resnet18(pretrained=True)
    num_features = model.fc.in_features
    model.fc = torch.nn.Linear(num_features, 10)  # CIFAR-10 has 10 classes
    model = model.to(device)  # Move model to GPU
    return model

# **SISA Algorithm**

In [140]:
# SISA Algorithm
def train_shard(model, data_loader, epochs):
    criterion = torch.nn.CrossEntropyLoss().to(device)
    optimizer = torch.optim.Adam(model.parameters(), lr=0.001)
    model.train()
    for epoch in range(epochs):
        for inputs, labels in data_loader:
            inputs, labels = inputs.to(device), labels.to(device)  # Move data to GPU
            optimizer.zero_grad()
            outputs = model(inputs)
            loss = criterion(outputs, labels)
            loss.backward()
            optimizer.step()
    return model

def sisa_training(dataset, S, R, epochs_list):
    shard_size = len(dataset) // S
    shards = random_split(dataset, [shard_size] * S)  # Creating shards
    models = []

    for shard in shards:
        slice_size = len(shard) // R
        slices = random_split(shard, [slice_size] * R)  # Creating slices
        model = create_model()
        for i, slice in enumerate(slices):
            data_loader = DataLoader(slice, batch_size=32, shuffle=True)
            model = train_shard(model, data_loader, epochs_list[i % len(epochs_list)])
        models.append(model)
    return models

# Evaluation and Aggregation
def evaluate_model(models, train_loader, test_loader):
    def metrics_report(loader):
        total_preds = []
        total_labels = []
        with torch.no_grad():
            for data, target in loader:
                data, target = data.to(device), target.to(device)
                model_outputs = [model(data).detach() for model in models]
                avg_output = torch.mean(torch.stack(model_outputs), dim=0)
                _, predicted = torch.max(avg_output, 1)
                total_preds.extend(predicted.cpu().tolist())
                total_labels.extend(target.cpu().tolist())

        accuracy = accuracy_score(total_labels, total_preds)
        precision = precision_score(total_labels, total_preds, average='macro')
        recall = recall_score(total_labels, total_preds, average='macro')
        f1 = f1_score(total_labels, total_preds, average='macro')
        true_binary = label_binarize(total_labels, classes=np.unique(total_labels))
        pred_binary = label_binarize(total_preds, classes=np.unique(total_labels))
        auroc = roc_auc_score(true_binary, pred_binary, multi_class='ovr')

        return accuracy, precision, recall, f1, auroc
    
    # Optionally evaluate on training data
    results = {}
    
    test_metrics = metrics_report(test_loader)
    results["Testing"] = test_metrics

    return results

# Function to display metrics in a beautiful table
def display_metrics(results):
    headers = ["Dataset", "Accuracy", "Precision", "Recall", "F1 Score", "AUROC"]
    table = []
    for dataset, metrics in results.items():
        row = [dataset] + [f"{metric:.4f}" for metric in metrics]
        table.append(row)
    print(tabulate(table, headers=headers, tablefmt="grid"))
    
# Function to unlearn the data
def unlearn_data(dataset, forget_indices):
    # Create a subset of the dataset excluding the forget_indices
    retain_indices = list(set(range(len(dataset))) - set(forget_indices))
    return Subset(dataset, retain_indices)

# Function to compute losses
def compute_losses(model, data_loader):
    model.to(device)
    model.eval()
    losses = []
    with torch.no_grad():
        for inputs, labels in data_loader:
            inputs, labels = inputs.to(device), labels.to(device)
            outputs = model(inputs)
            loss = cross_entropy(outputs, labels, reduction='none')
            losses.extend(loss.cpu().numpy())
    return np.array(losses)

# Function to compute cross-validation score of MIA
def mia_cross_val_score(train_losses, test_losses):
    labels = np.concatenate((np.ones(len(train_losses)), np.zeros(len(test_losses))))
    losses = np.concatenate((train_losses, test_losses)).reshape(-1, 1)
    clf = LogisticRegressionCV(cv=5, random_state=42, max_iter=1000).fit(losses, labels)
    scores = cross_val_score(clf, losses, labels, cv=5)
    return scores.mean()

#  Select 500 data points from a specific class in the training set
def select_and_poison_data_inplace(dataset, target_class, num_samples=500, block_size=3):
    class_indices = [i for i in range(len(dataset)) if dataset.targets[i] == target_class]
    selected_indices = random.sample(class_indices, num_samples)
    block_size = int(224 / 32) * block_size
    if block_size != 0:
        for idx in selected_indices:
            #print(dataset.targets[idx])
            img = dataset.data[idx]
            img = np.copy(img)  # To avoid modifying the original image
            # Randomly select a position for the 3x3 block
            x = random.randint(0, img.shape[0] - block_size)
            y = random.randint(0, img.shape[1] - block_size)
            img[x:x+block_size, y:y+block_size, :] = 0  # Turn the block to black
            dataset.data[idx] = img

    return selected_indices    

def predict_models(models, test_loader):
        all_predictions = []
        for model in models:
            model.eval()  # Set model to evaluation mode
            model_predictions = []
            for inputs, _ in test_loader:
                inputs = inputs.to(device)
                with torch.no_grad():
                    pred = model(inputs)
                model_predictions.append(pred.cpu())
            all_predictions.append(torch.cat(model_predictions))
        return all_predictions

def aggregate_average(models, test_loader):
        all_predictions = predict_models(models, test_loader)
        averaged_probabilities = torch.mean(torch.stack(all_predictions), dim=0)
        aggregated_predictions = torch.argmax(averaged_probabilities, dim=1)
        averaged_probabilities = F.softmax(averaged_probabilities, dim=1)
        return aggregated_predictions, averaged_probabilities
    
def poison_dataset(dataset, num_samples, target_class=0):
    # Prepare poisoned images from other classes
    poisoned_datasets = []

    for i in range(1, 10):
        if i != target_class:
            poisoned_indices = select_and_poison_data_inplace(dataset, target_class=i, num_samples=num_samples, block_size=3)
            poisoned_subset = Subset(dataset, poisoned_indices)
            poisoned_datasets.append(poisoned_subset)

    # Concatenate all poisoned subsets into one dataset
    poisoned_dataset = ConcatDataset(poisoned_datasets)
    
    return poisoned_dataset

# Calculate the Attack Success Rate (ASR)
def calculate_asr(dataset, model, num_samples, target_class=0):
    data_loader = DataLoader(dataset, batch_size=32, shuffle=False)

    aggregated_predictions, _ = aggregate_average(model, data_loader)

    misclassified_count = (aggregated_predictions == target_class).sum().item()
    total_count = len(dataset)
    
    asr = misclassified_count / total_count
    
    return asr

# Calculate the Attack Success Rate (ASR)
def calculate_poisoned_asr(dataset, model, num_samples, target_class=0):
    # Prepare poisoned images from other classes
    poisoned_dataset = poison_dataset(dataset, num_samples, target_class=0)
    poisoned_loader = DataLoader(poisoned_dataset, batch_size=32, shuffle=False)

    aggregated_predictions, _ = aggregate_average(model, poisoned_loader)

    misclassified_count = (aggregated_predictions == target_class).sum().item()
    total_count = len(poisoned_dataset)
    
    asr = misclassified_count / total_count
    
    return asr

# **Training Phase**

## **Simulation Question 1.**

In [95]:
# Number of Shards and Slices
S_values = [5, 10, 20]  # List of S values
R_values = [5, 10, 20]  # List of R values
epochs = [2, 2, 5]  # Number of epochs for training

best_accuracy = 0        # Variable to track the best accuracy
best_model = None        # Variable to store the best model
best_train_loader = None # Variable to store the best train loader
best_test_loader = None  # Variable to store the best test loader
best_result = {}         # Variable to store the best result

for S in S_values:
    for R in R_values:
        print(f"\nTraining with S={S}, R={R}")
        
        # Train the model with the given S, R, and epochs
        model = sisa_training(train_set, S, R, epochs)
        
        # Evaluate the trained model
        train_loader = DataLoader(train_set, batch_size=32, pin_memory=True, num_workers=4)
        test_loader = DataLoader(test_set, batch_size=32, pin_memory=True, num_workers=4)
        result = evaluate_model(model, train_loader, test_loader)
        
        # Display evaluation metrics
        display_metrics(result)
        
        # Calculate the accuracy score
        accuracy = result["Testing"][0]
        
        # Update the best model if the current one is better
        if accuracy > best_accuracy:
            best_accuracy = accuracy
            best_model = model
            best_train_loader = train_loader
            best_test_loader = test_loader
            best_result = result



Training with S=5, R=5

+-----------+------------+-------------+----------+------------+---------+

| Dataset   |   Accuracy |   Precision |   Recall |   F1 Score |   AUROC |

+===========+============+=============+==========+============+=========+

| Testing   |     0.8449 |      0.8452 |   0.8449 |     0.8445 |  0.9138 |

+-----------+------------+-------------+----------+------------+---------+



Training with S=5, R=10

+-----------+------------+-------------+----------+------------+---------+

| Dataset   |   Accuracy |   Precision |   Recall |   F1 Score |   AUROC |

+===========+============+=============+==========+============+=========+

| Testing   |     0.8193 |      0.8246 |   0.8193 |     0.8208 |  0.8996 |

+-----------+------------+-------------+----------+------------+---------+



Training with S=5, R=20

+-----------+------------+-------------+----------+------------+---------+

| Dataset   |   Accuracy |   Precision |   Recall |   F1 Score |   AUROC |

+=======

| S  | R  | Metric   | Train Accuracy | Train Precision | Train Recall | Train F1 Score | Train AUROC | Test Accuracy | Test Precision | Test Recall | Test F1 Score | Test AUROC |
|----|----|----------|----------------|-----------------|--------------|----------------|-------------|---------------|----------------|-------------|---------------|------------|
| 5  | 5  |          | 0.86504        | 0.86495         | 0.86504      | 0.86495        | 0.92502     | 0.8412        | 0.84080        | 0.8412      | 0.84087       | 0.91178    |
| 5  | 10 |          | 0.84216        | 0.84290         | 0.84216      | 0.84224        | 0.91231     | 0.8225        | 0.82275        | 0.8225      | 0.82225       | 0.90139    |
| 5  | 20 |          | 0.81016        | 0.81049         | 0.81016      | 0.80973        | 0.89453     | 0.7939        | 0.79380        | 0.7939      | 0.79296       | 0.88550    |
| 10 | 5  |          | 0.8186         | 0.81859         | 0.8186       | 0.81781        | 0.89922     | 0.8093        | 0.80872        | 0.8093      | 0.80807       | 0.89406    |
| 10 | 10 |          | 0.78334        | 0.78686         | 0.78334      | 0.78341        | 0.87963     | 0.7738        | 0.77626        | 0.7738      | 0.77350       | 0.87433    |
| 10 | 20 |          | 0.7493         | 0.75252         | 0.7493       | 0.74963        | 0.86072     | 0.7357        | 0.73852        | 0.7357      | 0.73559       | 0.85317    |
| 20 | 5  |          | 0.7592         | 0.75386         | 0.7592       | 0.75225        | 0.86273     | 0.7453        | 0.74536        | 0.7453      | 0.74385       | 0.8585    |
| 20 | 10 |          | 0.7161        | 0.71762         | 0.7161      | 0.71458        | 0.84230     | 0.7086        | 0.70984        | 0.7086      | 0.70657       | 0.83811    |
| 20 | 20 |          | 0.6873         | 0.68742         | 0.6873       | 0.68731        | 0.7976     | 0.6534        | 0.65431        | 0.6534      | 0.65423       | 0.7765    |

## Proposed Aggregation Methods for SISA Training

In the SISA training algorithm, models are trained independently on different shards of the dataset. To get a final prediction, outputs from all trained models are aggregated. This aggregation plays a crucial role in forming a comprehensive model that integrates learned patterns across all shards.

## Aggregation Technique

### Average Aggregation

- **Description**: The implemented aggregation method computes the average of the outputs from all shard-specific models. For each input batch, the models generate logits (pre-softmax scores), and these logits are averaged across all models before applying a final softmax for classification.
- **Implementation**:
  - **Step 1**: Collect outputs from each model for the current batch.
  - **Step 2**: Stack all outputs to form a tensor where each model contributes its output logits.
  - **Step 3**: Compute the mean across these logits to get an average logit.
  - **Step 4**: Apply the softmax function to convert these average logits into probabilities.
  - **Step 5**: Determine the predicted class by selecting the class with the highest probability.
- **Advantages**:
  - Reduces model variance as it incorporates knowledge from multiple independently trained models.
  - Can improve generalization by smoothing out predictions.

### Metrics Evaluation

- After aggregation, the final predictions are evaluated against the true labels to compute performance metrics:
  - **Accuracy**
  - **Precision**
  - **Recall**
  - **F1 Score**
  - **AUROC** (Area Under the Receiver Operating Characteristic curve)
- These metrics provide a comprehensive assessment of model performance post-aggregation, highlighting the effectiveness of the aggregation strategy in leveraging distributed learning.

## Usage Scenario

This aggregation method is particularly useful in scenarios where training data is large and diverse, making it beneficial to train on partitions and later integrate learning. It's also relevant for privacy-preserving learning where data cannot be centralized due to privacy concerns or regulations.


## **Simulation Question 2.**

In [96]:
# Number of Shards and Slices
S_values = [5, 10]
R_values = [5, 10]
epochs = [2, 2, 5]  # Simplified for demonstration

best_accuracy_after_unlearning = 0        # Variable to track the best accuracy
best_model_after_unlearning = None        # Variable to store the best model
best_train_loader_after_unlearning = None # Variable to store the best train loader
best_test_loader_after_unlearning = None  # Variable to store the best test loader
best_result_after_unlearning = {}         # Variable to store the best result

# Select 500 random samples to forget
forget_indices = random.sample(range(len(train_set)), 500)

# Unlearn 500 samples
unlearned_train_set = unlearn_data(train_set, forget_indices)

for S in S_values:
    for R in R_values:
        print(f"\nTraining with S={S}, R={R}")
        
        # Train the model with the given S, R, and epochs
        model_after_unlearning = sisa_training(unlearned_train_set, S, R, epochs)
        
        # Evaluate the trained model
        train_loader_after_unlearning = DataLoader(unlearned_train_set, batch_size=32, pin_memory=True, num_workers=4)
        test_loader_after_unlearning = DataLoader(unlearned_train_set, batch_size=32, pin_memory=True, num_workers=4)
        result_after_unlearning = evaluate_model(model_after_unlearning, train_loader_after_unlearning, test_loader_after_unlearning)
        
        # Display evaluation metrics
        display_metrics(result_after_unlearning)
        
        # Calculate the accuracy score
        accuracy_after_unlearning = result_after_unlearning["Testing"][0]
        
        # Update the best model if the current one is better
        if accuracy_after_unlearning > best_accuracy_after_unlearning:
            best_accuracy_after_unlearning = accuracy_after_unlearning
            best_mode_after_unlearningl = model_after_unlearning
            best_train_loader_after_unlearning = train_loader_after_unlearning
            best_test_loader_after_unlearning = test_loader_after_unlearning
            best_result_after_unlearning = result_after_unlearning



Training with S=5, R=5

+-----------+------------+-------------+----------+------------+---------+

| Dataset   |   Accuracy |   Precision |   Recall |   F1 Score |   AUROC |

+===========+============+=============+==========+============+=========+

| Testing   |     0.8597 |       0.863 |   0.8597 |       0.86 |   0.922 |

+-----------+------------+-------------+----------+------------+---------+



Training with S=5, R=10

+-----------+------------+-------------+----------+------------+---------+

| Dataset   |   Accuracy |   Precision |   Recall |   F1 Score |   AUROC |

+===========+============+=============+==========+============+=========+

| Testing   |     0.8401 |      0.8418 |   0.8401 |     0.8401 |  0.9112 |

+-----------+------------+-------------+----------+------------+---------+



Training with S=10, R=5

+-----------+------------+-------------+----------+------------+---------+

| Dataset   |   Accuracy |   Precision |   Recall |   F1 Score |   AUROC |

+=======

In [97]:
# Select 500 random samples to forget
retain_indices = list(set(range(len(train_set))) - set(forget_indices))

retain_set = Subset(train_set, retain_indices)
forget_set = Subset(train_set, forget_indices)
test_sample_indices_1 = random.sample(range(len(test_set)), 500)
test_sample_set_1 = Subset(test_set, test_sample_indices_1)
test_sample_indices_2 = random.sample(range(len(test_set)), 500)
test_sample_set_2 = Subset(test_set, test_sample_indices_2)

retain_loader = DataLoader(retain_set, batch_size=32, shuffle=False)
forget_loader = DataLoader(forget_set, batch_size=32, shuffle=False)
test_sample_loader_1 = DataLoader(test_sample_set_1, batch_size=32, shuffle=False)
test_sample_loader_2 = DataLoader(test_sample_set_2, batch_size=32, shuffle=False)

In [100]:
# Assuming models and models_after_unlearning are loaded or defined elsewhere
# Compute losses for the trained model
# retain_losses = np.concatenate([compute_losses(model,  DataLoader(retain_set, batch_size=32, shuffle=False)) for model in best_model])
# forget_losses = np.concatenate([compute_losses(model, DataLoader(forget_set, batch_size=32, shuffle=False)) for model in best_model])
# test_losses_1 = np.concatenate([compute_losses(model, DataLoader(test_sample_set_1, batch_size=32, shuffle=False)) for model in best_model])
# test_losses_2 = np.concatenate([compute_losses(model, DataLoader(test_sample_set_2, batch_size=32, shuffle=False)) for model in best_model])

# mia_cross_val_score12 = mia_cross_val_score(retain_losses, test_losses_1)
# mia_cross_val_score34 = mia_cross_val_score(forget_losses, test_losses_2)

# MIA scores for the trained model
print("MIA scores for the trained model:")
print("Datasets 1 and 2:", mia_cross_val_score12)
print("Datasets 3 and 4:", mia_cross_val_score34)

# Compute losses for the unlearned model
# retain_losses_after = np.concatenate([compute_losses(model, DataLoader(retain_set, batch_size=32, shuffle=False)) for model in models_after_unlearning])
# forget_losses_after = np.concatenate([compute_losses(model, DataLoader(forget_set, batch_size=32, shuffle=False)) for model in models_after_unlearning])
# test_losses_1_after = np.concatenate([compute_losses(model, DataLoader(test_sample_set_1, batch_size=32, shuffle=False)) for model in models_after_unlearning])
# test_losses_2_after = np.concatenate([compute_losses(model, DataLoader(test_sample_set_2, batch_size=32, shuffle=False)) for model in models_after_unlearning])

# mia_cross_val_score_after12 = mia_cross_val_score(retain_losses_after, test_losses_1_after)
# mia_cross_val_score_after34 = mia_cross_val_score(forget_losses_after, test_losses_2_after)

# MIA scores for the unlearned model
print("MIA scores for the unlearned model:")
print("Datasets 1 and 2:", mia_cross_val_score_after12)
print("Datasets 3 and 4:", mia_cross_val_score_after34)

MIA scores for the trained model:
Datasets 1 and 2: 0.5302
Datasets 3 and 4: 0.5348
MIA scores for the unlearned model:
Datasets 1 and 2: 0.5276
Datasets 3 and 4: 0.5308


In [139]:
# Poisening the Best model; S = 5, R = 5
poisoned_model = best_model

# Evaluate the poisoned model's performance on clean test data
display_metrics(best_result)

# Calculate the attack success rate (ASR)
asr = calculate_asr(test_set, poisoned_model, 100, 0)
print(f"Attack Success Rate (ASR) Before Poisoninng: {asr * 100:.2f}%")

# Calculate the attack success rate (ASR)
asr = calculate_poisoned_asr(test_set, poisoned_model, 100, 0)
print(f"Attack Success Rate (ASR) After Poisoninng: {asr * 100:.2f}%")

+-----------+------------+-------------+----------+------------+---------+

| Dataset   |   Accuracy |   Precision |   Recall |   F1 Score |   AUROC |

+===========+============+=============+==========+============+=========+

| Testing   |      0.838 |      0.8388 |    0.838 |     0.8373 |    0.91 |

+-----------+------------+-------------+----------+------------+---------+

Attack Success Rate (ASR) Before Poisoninng: 12.58%

Attack Success Rate (ASR) After Poisoninng: 64.44%


In [135]:
# Poisening the Best model; S = 5, R = 5
poisoned_model_after_unlearning = best_mode_after_unlearningl

# Evaluate the poisoned model's performance on clean test data
display_metrics(best_result_after_unlearning)

# Calculate the attack success rate (ASR)
asr = calculate_asr(test_set, poisoned_model_after_unlearning, 100, 0)
print(f"Attack Success Rate (ASR) with unlearning Before Poisoninng: {asr * 100:.2f}%")

# Calculate the attack success rate (ASR)
asr = calculate_poisoned_asr(test_set, poisoned_model_after_unlearning, 100, 0)
print(f"Attack Success Rate (ASR) with unlearning After Poisoninng: {asr * 100:.2f}%")

+-----------+------------+-------------+----------+------------+---------+

| Dataset   |   Accuracy |   Precision |   Recall |   F1 Score |   AUROC |

+===========+============+=============+==========+============+=========+

| Testing   |     0.8597 |       0.863 |   0.8597 |       0.86 |   0.922 |

+-----------+------------+-------------+----------+------------+---------+

Attack Success Rate (ASR) with unlearning Before Poisoninng: 12.11%

Attack Success Rate (ASR) with unlearning After Poisoninng: 3.33%
